In [1]:
!pip install pandas
!pip install pandas openpyxl
!pip install xlrd



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd 
import os
import numpy as np

In [3]:
def answer_one():
    file_name= os.path.join('Energy Indicators.xls')
    df = pd.read_excel(file_name, skiprows=17, skipfooter=38)
    pd.set_option('display.max_rows', None)  
    pd.set_option('display.max_columns', None) 
    energy = pd.DataFrame(df)
    
    del energy['Unnamed: 0']
    del energy['Unnamed: 1']

    energy.rename(columns = {'Unnamed: 2':'Country', 'Petajoules':'Energy Supply', 'Gigajoules':'Energy Supply per Capita', '%':'% Renewable'}, inplace = True) #'Energy Supply','Energy Supply per Capita', '% Renewable')

    def convert_petajoule_to_gigajoules(x):
        if x == '...':
            return x
        else:
            return x * 10**6

    energy['Energy Supply'] = energy['Energy Supply'].apply(convert_petajoule_to_gigajoules)

    energy[['Energy Supply', 'Energy Supply per Capita']] = np.where(
    energy[['Energy Supply', 'Energy Supply per Capita']] == '...', np.nan, energy[['Energy Supply', 'Energy Supply per Capita']])   

    def delete_number(x):
        return ' '.join([char for char in str(x) if not char.isdigit()])

    energy['Country'] = energy['Country'].apply(lambda x: ''.join([char for char in str(x) if not char.isdigit()]))

    def remove_text_in_brackets(input_string):
        result = []
        inside_brackets = False
    
        for char in input_string:
            if char == '(':
                inside_brackets = True
            elif char == ')':
                inside_brackets = False
            elif not inside_brackets:
                result.append(char)
    
        return ''.join(result)

    energy['Country'] = energy['Country'].apply(remove_text_in_brackets)

    dict = {'Republic of Korea': 'South Korea', 'United States of America': 'United States', 
        'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom', 'China, Hong Kong Special Administrative Region': 'Hong Kong'}

    for key, var in dict.items():
        energy['Country'] = energy['Country'].replace(key, var)

    file_name_csv = 'world_bank.csv'
    df_two = pd.read_csv(file_name_csv, sep = ',', header= None, skiprows=4, encoding='utf-8')
    GDP = pd.DataFrame(df_two)

    dict_two = {'Korea, Rep.': 'South Korea', 'Iran, Islamic Rep.': 'Iran', 
        'Hong Kong SAR, China': 'Hong Kong'}

    GDP[0] = GDP[0].replace(dict_two)
    GDP.columns = GDP.iloc[0]  
    GDP = GDP[1:]
    GDP.rename(columns = {'Country Name':'Country'}, inplace = True) 

    file_name_three= os.path.join('scimagojr.xlsx')
    df_three = pd.read_excel(file_name_three, skiprows=0, skipfooter=0)
    pd.set_option('display.max_rows', None)  
    pd.set_option('display.max_columns', None) 
    ScimEn = pd.DataFrame(df_three)

    energy.set_index('Country', inplace=True)
    GDP.set_index('Country', inplace=True)
    ScimEn.set_index('Country', inplace=True)

    old_column_value = 1960.0
    for i in range(1960, 2016, 1):
        GDP.rename(columns={old_column_value: int(old_column_value)}, inplace=True)
        old_column_value += 1

    GDP_selected = GDP.loc[:, 2006:2015]
    df_four = energy.merge(ScimEn, on="Country", how="inner")
    df_four = df_four.merge(GDP_selected, on="Country", how="inner")
    df_four = df_four.drop(columns=['Region'])
    df_filtered = df_four[df_four['Rank'] <= 15]
    df_filtered = df_filtered.sort_values(by='Rank')
        
    return df_filtered

In [4]:
a = answer_one()
a

,Energy Supply,Energy Supply per Capita,% Renewable,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,127191000000,93,19.754910,1,273437,272374,2336764,1615239,8.55,245,2.752119e+12,3.550328e+12,4.594337e+12,5.101691e+12,6.087192e+12,7.551546e+12,8.532185e+12,9.570471e+12,1.047562e+13,1.106157e+13
United States,90838000000,286,11.570980,2,175891,172431,2230544,724472,12.68,363,1.381558e+13,1.447423e+13,1.476986e+13,1.447807e+13,1.504897e+13,1.559973e+13,1.625397e+13,1.688068e+13,1.760814e+13,1.829502e+13
India,33195000000,26,14.969080,3,55082,53775,463165,162944,8.41,181,9.402599e+11,1.216736e+12,1.198895e+12,1.341888e+12,1.675616e+12,1.823052e+12,1.827638e+12,1.856722e+12,2.039126e+12,2.103588e+12
Japan,18984000000,149,10.232820,4,50523,50065,488062,119930,9.66,193,4.601663e+12,4.579751e+12,5.106679e+12,5.289493e+12,5.759072e+12,6.233147e+12,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12
United Kingdom,7920000000,124,10.600470,5,43389,42284,615670,111290,14.19,226,2.708442e+12,3.090510e+12,2.929412e+12,2.412840e+12,2.485483e+12,2.663806e+12,2.707090e+12,2.784854e+12,3.064708e+12,2.927911e+12
Germany,13261000000,165,17.901530,6,38739,38013,433148,95145,11.18,196,3.046309e+12,3.484057e+12,3.808786e+12,3.479801e+12,3.468154e+12,3.824829e+12,3.597897e+12,3.808086e+12,3.965801e+12,3.423568e+12
Russian Federation,30709000000,214,17.288680,7,36735,36560,115938,54993,3.16,90,9.899321e+11,1.299703e+12,1.660848e+12,1.222646e+12,1.524917e+12,2.045923e+12,2.208294e+12,2.292470e+12,2.059242e+12,1.363482e+12
Canada,10431000000,296,61.945430,8,33472,32863,568080,100953,16.97,227,1.319265e+12,1.468820e+12,1.552990e+12,1.374625e+12,1.617343e+12,1.793327e+12,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12
Italy,6530000000,109,33.667230,9,27983,26940,352993,87828,12.61,166,1.958564e+12,2.222524e+12,2.417508e+12,2.209484e+12,2.144936e+12,2.306974e+12,2.097929e+12,2.153226e+12,2.173256e+12,1.845428e+12
